In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
from seaborn import color_palette
import numpy as np
from util import load_results_from_dir

# sns.set_palette('rocket_r')
# sns.set_palette('hls')
# colors = ["#5B656F", "#A1B5D8", "#EDCB96", "#DB5461", "#171738", "#6C783B"]
# colors = ["#fd7f6f", "#7eb0d5", "#b2e061", "#bd7ebe", "#ffb55a", "#ffee65", "#beb9db", "#fdcce5", "#8bd3c7"]
# colors = ["#ffb3b8", "#fa4d56", "#9f1853", "#1192e8", "#009d9a", "#005d5d"]
colors = ["#f77e84", "#9f1853", "#1192e8", "#009d9a", "#012749", "#ACACAC"]
sns.set_palette(sns.color_palette(colors))

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        # "font.serif": ["Computer Modern Roman", "Times New Roman"] + plt.rcParams["font.serif"],
        "text.latex.preamble": r"\usepackage[T1]{fontenc} \usepackage[utf8]{inputenc} \usepackage{lmodern}",
    }
)
figsize=(3,2)
dpi = 120
parameters = ['scheduler', 'N_w', 'N_d', 'N_s', 'W_hat', 'pi', 'sigma', 'anchor_count', 'epsilon', 'int_feas_tol', 'v', 'r_charge', 'r_deplete', 'B_min', 'B_max', 'B_start', 'trial']

In [ ]:
rootdir = "../out/villalvernia.elrond/grid_search"
df = load_results_from_dir(rootdir)
df = df[lambda x: (x.scheduler == 'naivescheduler') | (x.pi.isin([np.inf, 61, 46, 31, 16, 8]))]
df = df[lambda x: x.trial == 1]
df = df.sort_values('experiment_timestamp', ascending=False).drop_duplicates(parameters)

# add extra columns
df['n_charged_windows_drones'] = df.charged_windows_drones.apply(lambda x: [len(v) for v in x])
df['n_waited_windows_drones'] = df.waited_windows_drones.apply(lambda x: [len(v) for v in x])
df['n_charged_windows_slowest'] = df.apply(lambda x: x.n_charged_windows_drones[np.argmax(x.execution_time_all)], axis=1)
df['n_waited_windows_slowest'] = df.apply(lambda x: x.n_waited_windows_drones[np.argmax(x.execution_time_all)], axis=1)

In [ ]:
print(f"Size of dataset: {df.shape[0]:,}\n")
df.sort_values(by='experiment_timestamp')[parameters + ['execution_time', 't_solve_mean', 't_solve_total', 'n_solves']]

# Plots

In [ ]:
label_translation = {
    'inf': '$\infty$',
    '8.0': '8',
    '16.0': '16',
    '31.0': '31', 
    '46.0': '46',
    '61.0': '61',
}

## Execution time

In [ ]:
cp = color_palette()
ymax = df.execution_time.max() * 1.1
order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].W_hat.unique())
hue_order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].pi.unique())

naive_execution_time = df[lambda x: x.scheduler == 'naivescheduler'].iloc[0].execution_time

for i, sigma in enumerate(sorted(set(df.sigma.unique()) - {0})):
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != "naivescheduler")]

    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    sns.barplot(data=df_plot, x='W_hat', y='execution_time', hue='pi', edgecolor='black', alpha=0.7, order=order, hue_order=hue_order, ax=ax, palette=cp)
    
    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, title="$\pi$", loc='right', edgecolor='white', framealpha=1)
    ax.axhline(y=naive_execution_time, color='red', linestyle=':')
    if i != 0:
        leg.remove()
    plt.xlabel("$\hat{N}_w$")
    plt.ylabel("Execution time (s)")
    plt.grid(axis='y')
    plt.ylim([0, ymax])
    # plt.title(f"$\sigma = {sigma}$")
        
    plt.savefig(f"../out/figures/grid_search/sigma{sigma}.execution_time.bar.pdf", bbox_inches='tight')
    plt.show()

## Scheduling time

### Total

In [ ]:
cp = color_palette()
ymax = df.t_solve_total.max() * 2
ymin = df.t_solve_total[lambda x: x>0].min() / 1.1
order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].W_hat.unique())
hue_order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].pi.unique())

for i, sigma in enumerate(sorted(set(df.sigma.unique()) - {0})):
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != "naivescheduler")]

    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    sns.barplot(data=df_plot, x='W_hat', y='t_solve_total', hue='pi', edgecolor='black', alpha=0.7, order=order, hue_order=hue_order, ax=ax, palette=cp)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, title="$\pi$", loc='right', edgecolor='white', framealpha=1)
    if i != 0:
        leg.remove()
    plt.xlabel("$\hat{N}_w$")
    plt.ylabel("Total scheduling time (s)")
    plt.grid(axis='y')
    plt.yscale('log')
    ax.set_ylim([ymin, ymax])
    # plt.title(f"$\sigma = {sigma}$")
    
    plt.savefig(f"../out/figures/grid_search/sigma{sigma}.scheduling_time_total.bar.pdf", bbox_inches='tight')
    plt.show()

### Average

In [ ]:
cp = color_palette()
ymax = df.t_solve_mean.max() * 2
ymin = df.t_solve_mean[lambda x: x>0].min() / 1.1
order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].W_hat.unique())
hue_order = sorted(df[lambda x: x.scheduler != 'naivescheduler'].pi.unique())

for i, sigma in enumerate(sorted(set(df.sigma.unique()) - {0})):
    print(sigma)
    df_plot = df[lambda x: (x.sigma == sigma) & (x.scheduler != "naivescheduler")]

    _, ax = plt.subplots(figsize=figsize, dpi=dpi)
    sns.barplot(data=df_plot, x='W_hat', y='t_solve_mean', hue='pi', edgecolor='black', alpha=0.7, order=order, hue_order=hue_order, ax=ax, palette=cp)

    handles, labels = ax.get_legend_handles_labels()
    new_labels = [label_translation[i] for i in labels]
    leg = ax.legend(handles, new_labels, title="$\pi$", loc='right', edgecolor='white', framealpha=1)
    if i != 0:
        leg.remove()
    
    plt.xlabel("$\hat{N}_w$")
    plt.ylabel("Mean scheduling time (s)")
    plt.grid(axis='y')
    ax.set_yscale('log')
    ax.set_ylim([ymin, ymax])
    # plt.title(f"$\sigma = {sigma}$")
    
    plt.savefig(f"../out/figures/grid_search/sigma{sigma}.scheduling_time_mean.bar.pdf", bbox_inches='tight')
    plt.show()